In [2]:
from s3prl.downstream.mdd.dataset import L2ArcticDataset
from s3prl.downstream.augment_utils.add_noise import AddNoise
from IPython.display import Audio, display
import torch
import torchaudio

2024-04-04 16:30:35.061468: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-04 16:30:35.466913: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-04 16:30:36.401309: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
dataset = L2ArcticDataset(
    "train",
    "/home/xt0r3-user/cambridge/partii/dissertation/s3prl/data/l2arctic_release_v5.0",
    "/home/xt0r3-user/cambridge/partii/dissertation/s3prl/s3prl/s3prl/downstream/mdd/data/",
)

Skipping malformatted TextGrid file: /home/xt0r3-user/cambridge/partii/dissertation/s3prl/data/l2arctic_release_v5.0/YDCK/annotation/arctic_a0209.TextGrid
Skipping malformatted TextGrid file: /home/xt0r3-user/cambridge/partii/dissertation/s3prl/data/l2arctic_release_v5.0/YDCK/annotation/arctic_a0272.TextGrid


In [3]:
def play_audio(waveform, sample_rate):
    waveform = waveform.numpy()
    display(Audio(waveform, rate=sample_rate))


audio = [
    torchaudio.functional.resample(dataset[0][0], orig_freq=44100, new_freq=16000),
    torchaudio.functional.resample(dataset[1][0], orig_freq=44100, new_freq=16000),
]

In [4]:
play_audio(audio[0], 16000)

In [6]:
augmentation = AddNoise(
    prob=1.0,
    snr_low=0,
    snr_high=0.4,
    device="cpu",
    seed=1337,
)
audio = torch.nn.utils.rnn.pad_sequence(audio, batch_first=True)
lens = torch.tensor([len(audio[0]), len(audio[1])], dtype=torch.long)
aug_audio =     augmentation(
        audio,
        lens,
    )
play_audio(
    aug_audio[0],
    sample_rate=16000,
)

audio - aug_audio

tensor([[ 1.4913e-06,  3.2584e-05, -2.9423e-05,  ...,  2.8078e-04,
         -3.6437e-05, -1.0176e-04],
        [-6.4237e-04, -7.4927e-03,  3.1364e-03,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]])